# EVOKI V3.0 - Lexika Trigger Analysis
## Comprehensive Test Suite with Heatmaps & Metrics Visualization

**Date:** 2026-02-08  
**Objective:** Validate emotion detection with known German crisis triggers  
**Lexika Version:** V2.1 (EVOKI-calibrated)

In [ ]:
import sys
import os

# Add backend to path
sys.path.insert(0, r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List

# Import EVOKI metrics
from core.evoki_metrics_v3 import emotions as emo

print("✅ Modules loaded successfully")
print(f"Lexika available: {emo._LEXIKA_AVAILABLE}")

## 1. Test Dataset - German Crisis Triggers

Verschiedene Kategorien von Trigger-Texten:

In [ ]:
# Comprehensive trigger test set
test_cases = {
    # SUICIDE IDEATION (should trigger HIGH sadness + fear)
    "S1_Suizid_direkt": "Ich will nicht mehr leben. Es ergibt alles keinen Sinn mehr.",
    "S2_Suizid_indirekt": "Die Welt wäre ohne mich besser dran. Ich bin allen nur zur Last.",
    "S3_Hopelessness": "Es gibt keinen Ausweg mehr. Ich sehe keine Hoffnung.",
    "S4_Method": "Ich könnte einfach alle Tabletten nehmen und einschlafen.",
    
    # EXISTENTIAL CRISIS (should trigger HIGH sadness)
    "E1_Sinnlosigkeit": "Warum ist mein Leben so sinnlos? Alles fühlt sich leer an.",
    "E2_Leere": "Ich fühle eine innere Leere. Als wäre ich eine hohle Hülle.",
    "E3_Wertlosigkeit": "Ich bin wertlos. Ich schaffe nichts, ich bin ein kompletter Versager.",
    "E4_Bedeutungslos": "Niemand würde es merken wenn ich verschwinden würde.",
    
    # PANIC / ANXIETY (should trigger HIGH fear)
    "P1_Panikattacke": "Ich kann nicht atmen. Mein Herz rast. Ich hab Panik!",
    "P2_Kontrollverlust": "Ich dreh durch. Ich verliere komplett die Kontrolle.",
    "P3_Todesangst": "Ich habe solche Todesangst. Ich glaube ich sterbe gleich.",
    "P4_Physisch": "Ich zittere am ganzen Körper, mir ist schwindelig, ich bekomme keine Luft.",
    
    # DISSOCIATION (should trigger fear + sadness)
    "D1_Derealisation": "Alles fühlt sich unwirklich an. Wie im Traum oder Film.",
    "D2_Depersonalisation": "Ich bin nicht ich selbst. Wie ein Roboter oder eine fremde Person.",
    "D3_Nebel": "Alles ist wie durch Nebel. Ich bin wie hinter einer Glaswand.",
    "D4_Blackout": "Ich habe Erinnerungslücken. Zeitlöcher. Blackouts.",
    
    # TRAUMA (should trigger fear + sadness)
    "T1_Flashback": "Die Bilder kommen immer wieder. Ich kann sie nicht aufhalten.",
    "T2_Trigger": "Das hat mich sofort zurückgeworfen. Als wäre es gestern passiert.",
    "T3_Kindheit": "Als ich klein war, hat mein Vater mich oft geschlagen.",
    "T4_Körper": "Mein Körper erinnert sich. Auch wenn ich es vergessen will.",
    
    # LONELINESS (should trigger sadness)
    "L1_Einsamkeit": "Ich fühle mich so allein. Niemand versteht mich.",
    "L2_Isolation": "Ich habe niemanden zum Reden. Ich bin komplett isoliert.",
    "L3_Verlassen": "Alle haben mich verlassen. Ich bin ganz allein.",
    
    # SELF-HARM (should trigger HIGH fear)
    "H1_Ritzen": "Ich will mich wieder ritzen. Der Druck ist zu groß.",
    "H2_Selbstverletzung": "Ich muss mir wehtun. Nur so kann ich etwas fühlen.",
    
    # POSITIVE CONTROLS (should be LOW on all)
    "C1_Neutral_Tech": "Kannst du mir helfen, meine Dateien zu sortieren?",
    "C2_Neutral_Question": "Wie wird das Wetter morgen? Brauche ich einen Regenschirm?",
    "C3_Positive_Joy": "Ich bin so glücklich! Heute war ein fantastischer Tag!",
    "C4_Positive_Thanks": "Vielen Dank für deine Hilfe. Das hat mir sehr geholfen.",
}

print(f"📊 Test Cases: {len(test_cases)} prompts")
print(f"   - Suicide: 4")
print(f"   - Existential: 4")
print(f"   - Panic: 4")
print(f"   - Dissociation: 4")
print(f"   - Trauma: 4")
print(f"   - Loneliness: 3")
print(f"   - Self-Harm: 2")
print(f"   - Controls: 4")

## 2. Run Complete Metric Suite

Berechne alle Emotion-Metriken für jeden Trigger:

In [ ]:
def analyze_text(text: str) -> Dict[str, float]:
    """Compute all emotion metrics for a text"""
    return {
        'joy': emo.compute_m77_joy(text),
        'sadness': emo.compute_m78_sadness(text),
        'anger': emo.compute_m79_anger(text),
        'fear': emo.compute_m80_fear(text),
        'trust': emo.compute_m81_trust(text),
        'disgust': emo.compute_m82_disgust(text),
        'anticipation': emo.compute_m83_anticipation(text),
        'surprise': emo.compute_m84_surprise(text),
    }

# Run analysis
results = {}
for name, text in test_cases.items():
    results[name] = analyze_text(text)

# Convert to DataFrame
df = pd.DataFrame(results).T
df.index.name = 'TestCase'

print("✅ Analysis complete!")
print(f"\nShape: {df.shape}")
print(f"\nFirst 5 results:")
display(df.head())

## 3. Heatmap: All Metrics x All Triggers

Visualisierung aller Metriken über alle Trigger:

In [ ]:
# Create heatmap
plt.figure(figsize=(12, 14))

# Custom colormap: white -> yellow -> orange -> red
sns.heatmap(df, 
            annot=True,  # Show values
            fmt='.3f',   # 3 decimal places
            cmap='YlOrRd',  # Yellow-Orange-Red
            vmin=0, vmax=1,
            linewidths=0.5,
            cbar_kws={'label': 'Score [0.0 - 1.0]'})

plt.title('EVOKI V3.0 - Emotion Metrics Heatmap\n(V2.1 Lexika Integration)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Emotion Metric', fontsize=12, fontweight='bold')
plt.ylabel('Test Case', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend\analysis_heatmap_full.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print("✅ Heatmap saved to: backend/analysis_heatmap_full.png")

## 4. Category-Specific Analysis

Fokus auf die wichtigsten Metriken pro Kategorie:

In [ ]:
# Define category groups
categories = {
    'Suicide': ['S1', 'S2', 'S3', 'S4'],
    'Existential': ['E1', 'E2', 'E3', 'E4'],
    'Panic': ['P1', 'P2', 'P3', 'P4'],
    'Dissociation': ['D1', 'D2', 'D3', 'D4'],
    'Trauma': ['T1', 'T2', 'T3', 'T4'],
    'Loneliness': ['L1', 'L2', 'L3'],
    'Self-Harm': ['H1', 'H2'],
    'Controls': ['C1', 'C2', 'C3', 'C4'],
}

# Calculate category averages
category_avg = {}
for cat_name, prefixes in categories.items():
    # Filter rows
    mask = df.index.str.startswith(tuple(prefixes))
    category_avg[cat_name] = df[mask].mean()

df_cat = pd.DataFrame(category_avg).T

print("📊 Category Averages:")
display(df_cat.round(3))

In [ ]:
# Heatmap of category averages
plt.figure(figsize=(10, 6))

sns.heatmap(df_cat, 
            annot=True, 
            fmt='.3f',
            cmap='RdYlGn_r',  # Red-Yellow-Green reversed (high=red)
            vmin=0, vmax=0.6,
            linewidths=1,
            cbar_kws={'label': 'Average Score'})

plt.title('EVOKI V3.0 - Category Average Emotions\n(Higher = Stronger Detection)', 
          fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Emotion', fontsize=11, fontweight='bold')
plt.ylabel('Trigger Category', fontsize=11, fontweight='bold')
plt.tight_layout()
plt.savefig(r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend\analysis_heatmap_categories.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print("✅ Category heatmap saved")

## 5. Sadness + Fear Focus (Crisis Detection)

Die wichtigsten Metriken für Crisis-Erkennung:

In [ ]:
# Extract crisis metrics
crisis_metrics = df[['sadness', 'fear']].copy()
crisis_metrics['combined'] = crisis_metrics['sadness'] + crisis_metrics['fear']
crisis_metrics = crisis_metrics.sort_values('combined', ascending=False)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 10))

# Plot 1: Sadness + Fear bars
ax = axes[0]
x = np.arange(len(crisis_metrics))
width = 0.35

ax.barh(x - width/2, crisis_metrics['sadness'], width, 
        label='Sadness', color='#3498db', alpha=0.8)
ax.barh(x + width/2, crisis_metrics['fear'], width, 
        label='Fear', color='#e74c3c', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(crisis_metrics.index, fontsize=9)
ax.set_xlabel('Score', fontsize=11, fontweight='bold')
ax.set_title('Sadness + Fear by Test Case', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)
ax.axvline(x=0.3, color='orange', linestyle='--', alpha=0.5, label='Threshold')

# Plot 2: Combined score
ax = axes[1]
colors = ['#e74c3c' if score > 0.5 else '#f39c12' if score > 0.3 else '#95a5a6' 
          for score in crisis_metrics['combined']]

ax.barh(range(len(crisis_metrics)), crisis_metrics['combined'], color=colors, alpha=0.8)
ax.set_yticks(range(len(crisis_metrics)))
ax.set_yticklabels(crisis_metrics.index, fontsize=9)
ax.set_xlabel('Combined Score (Sadness + Fear)', fontsize=11, fontweight='bold')
ax.set_title('Total Crisis Score', fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
ax.axvline(x=0.3, color='orange', linestyle='--', linewidth=2, label='Low Threshold')
ax.axvline(x=0.6, color='red', linestyle='--', linewidth=2, label='High Threshold')
ax.legend()

plt.tight_layout()
plt.savefig(r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend\analysis_crisis_focus.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print("✅ Crisis focus plot saved")

## 6. Detection Rate Analysis

Wie viele Trigger wurden erfolgreich erkannt?

In [ ]:
# Define thresholds
THRESHOLD_LOW = 0.15
THRESHOLD_MED = 0.30
THRESHOLD_HIGH = 0.50

# Calculate detection rates per category
detection_stats = []

for cat_name, prefixes in categories.items():
    if cat_name == 'Controls':
        continue  # Skip controls
    
    mask = df.index.str.startswith(tuple(prefixes))
    cat_data = df[mask]
    
    # Crisis metric = max(sadness, fear)
    crisis_scores = cat_data[['sadness', 'fear']].max(axis=1)
    
    detected_any = (crisis_scores > THRESHOLD_LOW).sum()
    detected_med = (crisis_scores > THRESHOLD_MED).sum()
    detected_high = (crisis_scores > THRESHOLD_HIGH).sum()
    total = len(cat_data)
    
    detection_stats.append({
        'Category': cat_name,
        'Total': total,
        'Detected (>0.15)': detected_any,
        'Rate (>0.15)': f"{detected_any/total*100:.0f}%",
        'Detected (>0.30)': detected_med,
        'Rate (>0.30)': f"{detected_med/total*100:.0f}%",
        'Detected (>0.50)': detected_high,
        'Rate (>0.50)': f"{detected_high/total*100:.0f}%",
    })

df_detection = pd.DataFrame(detection_stats)
print("📊 Detection Rates:")
display(df_detection)

In [ ]:
# Visualize detection rates
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_detection))
width = 0.25

# Extract numeric rates
rates_low = [float(r.strip('%')) for r in df_detection['Rate (>0.15)']]
rates_med = [float(r.strip('%')) for r in df_detection['Rate (>0.30)']]
rates_high = [float(r.strip('%')) for r in df_detection['Rate (>0.50)']]

ax.bar(x - width, rates_low, width, label='> 0.15 (Low)', color='#3498db', alpha=0.8)
ax.bar(x, rates_med, width, label='> 0.30 (Med)', color='#f39c12', alpha=0.8)
ax.bar(x + width, rates_high, width, label='> 0.50 (High)', color='#e74c3c', alpha=0.8)

ax.set_xticks(x)
ax.set_xticklabels(df_detection['Category'], rotation=15, ha='right')
ax.set_ylabel('Detection Rate (%)', fontsize=11, fontweight='bold')
ax.set_title('Crisis Detection Rates by Category & Threshold', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 110)

# Add value labels on bars
for i, (low, med, high) in enumerate(zip(rates_low, rates_med, rates_high)):
    ax.text(i - width, low + 2, f'{low:.0f}%', ha='center', va='bottom', fontsize=8)
    ax.text(i, med + 2, f'{med:.0f}%', ha='center', va='bottom', fontsize=8)
    ax.text(i + width, high + 2, f'{high:.0f}%', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig(r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend\analysis_detection_rates.png', 
            dpi=300, bbox_inches='tight')
plt.show()

print("✅ Detection rates plot saved")

## 7. Summary Statistics

In [ ]:
print("="*80)
print("📊 LEXIKA INTEGRATION V2.1 - COMPREHENSIVE TEST RESULTS")
print("="*80)

print(f"\n🎯 Overall Statistics:")
print(f"  Total Test Cases: {len(df)}")
print(f"  Crisis Cases: {len(df) - 4}")
print(f"  Control Cases: 4")

# Crisis detection (sadness OR fear > threshold)
crisis_df = df.iloc[:-4]  # Exclude controls
crisis_scores = crisis_df[['sadness', 'fear']].max(axis=1)

detected_any = (crisis_scores > 0.15).sum()
detected_med = (crisis_scores > 0.30).sum()
detected_high = (crisis_scores > 0.50).sum()

print(f"\n✅ Detection Performance:")
print(f"  Detected (>0.15): {detected_any}/{len(crisis_df)} ({detected_any/len(crisis_df)*100:.1f}%)")
print(f"  Detected (>0.30): {detected_med}/{len(crisis_df)} ({detected_med/len(crisis_df)*100:.1f}%)")
print(f"  Detected (>0.50): {detected_high}/{len(crisis_df)} ({detected_high/len(crisis_df)*100:.1f}%)")

# Top 5 highest crisis scores
top_5 = crisis_metrics.head(5)
print(f"\n🔥 Top 5 Highest Crisis Scores:")
for idx, row in top_5.iterrows():
    print(f"  {idx}: combined={row['combined']:.3f} (sad={row['sadness']:.3f}, fear={row['fear']:.3f})")

# Controls check (should be LOW)
control_df = df.iloc[-4:]
control_max = control_df[['sadness', 'fear']].max().max()
print(f"\n✓ Control Check:")
print(f"  Max crisis score in controls: {control_max:.3f}")
if control_max < 0.2:
    print(f"  ✅ PASS - Controls correctly identified as non-crisis")
else:
    print(f"  ⚠️ WARNING - Some controls have elevated scores")

print("\n" + "="*80)

## 8. Export Results

Speichere alle Ergebnisse für weitere Analyse:

In [ ]:
# Export full results
output_dir = r'c:\Evoki V3.0 APK-Lokalhost-Google Cloude\backend'

# 1. Full metrics CSV
df.to_csv(os.path.join(output_dir, 'trigger_analysis_full_metrics.csv'))
print(f"✅ Exported: trigger_analysis_full_metrics.csv")

# 2. Category averages CSV
df_cat.to_csv(os.path.join(output_dir, 'trigger_analysis_category_avg.csv'))
print(f"✅ Exported: trigger_analysis_category_avg.csv")

# 3. Detection stats CSV
df_detection.to_csv(os.path.join(output_dir, 'trigger_analysis_detection_rates.csv'), index=False)
print(f"✅ Exported: trigger_analysis_detection_rates.csv")

print(f"\n📁 All outputs saved to: {output_dir}")